# Import Libraries

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop

In [3]:
import splitfolders

In [4]:
splitfolders.ratio("dataset\Mushrooms", output="dataset\dataSplit",
    seed=1337, ratio=(.7, .2, .1), group_prefix=None, move=False) # default values

# Load Data

In [2]:
trainPath = r'dataset\dataSplit\train'
validationPath = r'dataset\dataSplit\val'
testPath = r'dataset\dataSplit\test'

In [3]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Data Augmentation

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=0.45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

#Train
train_generator = train_datagen.flow_from_directory(
                                            trainPath,
                                                    target_size=(300, 300),
                                                    batch_size=64,
                                                    shuffle=True,
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                            )

#Validation
validation_generator=val_datagen.flow_from_directory(
                                            validationPath,
                                                    target_size=(300,300),
                                                    batch_size=64,
                                                    shuffle=True,
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                                )

#Test
test_generator=val_datagen.flow_from_directory(
                                            testPath,
                                                    target_size=(300,300),
                                                    batch_size=64,
                                                    shuffle=True,
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                                )

Found 3781 images belonging to 9 classes.
Found 1078 images belonging to 9 classes.
Found 549 images belonging to 9 classes.


# EDA

In [ ]:
# class names
class_names = os.listdir(trainPath)
print('All category : ',class_names)

In [ ]:
#Sampling Dataset
image ,label = next(iter(test_generator))

plt.figure(figsize=(20,10))
for i in range(10) :
  plt.subplot(2,5,i+1)
  plt.imshow(image[i])
  plt.title(class_names[np.argmax(label[i])])
  plt.axis("off")

# Modelling

In [5]:
# Callbacks
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss',  
                                    factor=0.4, patience=3, 
                                    verbose=1, mode='min', 
                                    min_delta=0.0001, min_lr=0,
                                    restore_best_weights=True)

# EfficientNet-B7

In [6]:
from efficientnet.tfkeras import EfficientNetB7

pre_trained_model = EfficientNetB7(
    include_top=False,
    weights="imagenet",
    input_shape=(300, 300, 3),
    pooling='avg',
)

In [7]:
for layer in pre_trained_model.layers:
    layer.trainable = False


last_layer = pre_trained_model.get_layer('avg_pool')
last_output = last_layer.output


x = layers.Flatten()(last_output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)              
x = layers.Dense (9, activation='softmax')(x)           

model_B7 = Model( pre_trained_model.input, x) 

model_B7.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

# Model Training

In [10]:
history = model_B7.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 20,
            callbacks=[reduceLROnPlat],
            verbose = 1)  

Epoch 1/20
60/60 [==============================] - 750s 13s/step - loss: 1.2169 - accuracy: 0.5596 - val_loss: 0.9514 - val_accuracy: 0.6612 - lr: 0.0010
Epoch 2/20
60/60 [==============================] - 739s 12s/step - loss: 1.1102 - accuracy: 0.5975 - val_loss: 1.0509 - val_accuracy: 0.6029 - lr: 0.0010
Epoch 3/20
60/60 [==============================] - 749s 12s/step - loss: 0.9963 - accuracy: 0.6379 - val_loss: 0.8116 - val_accuracy: 0.7086 - lr: 0.0010
Epoch 4/20
60/60 [==============================] - 724s 12s/step - loss: 0.9894 - accuracy: 0.6406 - val_loss: 0.7254 - val_accuracy: 0.7614 - lr: 0.0010
Epoch 5/20
60/60 [==============================] - 729s 12s/step - loss: 0.9440 - accuracy: 0.6570 - val_loss: 0.6519 - val_accuracy: 0.7796 - lr: 0.0010
Epoch 6/20
60/60 [==============================] - 767s 13s/step - loss: 0.9222 - accuracy: 0.6620 - val_loss: 0.6588 - val_accuracy: 0.7559 - lr: 0.0010
Epoch 7/20
60/60 [==============================] - 778s 13s/step - lo

# Evaluate Test Set

In [ ]:
B7_eval = model_B7.evaluate(test_generator,verbose=0)
B7_acc = round(B7_eval[1],2) * 100
B7_acc

# Saved Model

In [ ]:
model_B7.save('saved-models/model_EfficientNetB7.h5')

# Confusion Matrix

In [ ]:
y_test = []
y_pred = []

for i in range(len(test_generator)) :
    x,y = test_generator.next()
    for j in range(len(y)) :
        y_test.append(class_names[tf.argmax(y[j])])
        y_pred.append(class_names[np.argmax(model_B3.predict(x[j][None,...],verbose=0))])

In [ ]:
fig, ax = plt.subplots(figsize=(18, 13),dpi=150)
cm = confusion_matrix(y_test, y_pred)

colors = ["#393E46","#393E46"]
colormap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)

ax.text(0,-0.3,'Confusion Matrix ',fontfamily='serif',fontsize=15,fontweight='bold')
ax.text(0,-0.12,'EfficientB7 Model',fontfamily='serif',fontsize=12,fontweight='light')
sns.heatmap(ax=ax, data=cm,
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=1.5, annot=True,cmap=colormap, 
            annot_kws={"fontsize":12},fmt='')

ax.set_xticklabels(class_names)
ax.set_yticklabels(class_names)

ax.set_ylabel("Actual Label",fontsize=10)
ax.set_xlabel("Predicted Label",fontsize=10)

from matplotlib.patches import Rectangle
for i in range(0,8):
    ax.add_patch(Rectangle((i, i), 1, 1, fill=True,color='#00ADB5'))

plt.show()

# Prediction Image

In [ ]:
from keras.models import load_model
import keras.utils as image
import matplotlib.pyplot as plt
import numpy as np

model = load_model('Saved_Models/model_EfficientNetB7.h5')

In [ ]:
B7_eval = model_B7.evaluate(test_generator,verbose=0)
B7_acc = round(B7_eval[1],2) * 100
B7_acc

In [ ]:
img_path = ""
CLASS_NAMES = ['Agglonema', 'Alocasia', 'Gelombang Cinta', 'Janda Bolong', 'Lidah Mertua', 'Lili Paris', 'Pucuk Merah', 'Suplir']

img = image.load_img(img_path, target_size=(275,275))
imgplot = plt.imshow(img)
x = image.img_to_array(img)
x = x/255.0
x = np.expand_dims(x, axis=0)
images = np.vstack([x])

classes = model.predict(images, batch_size=10,verbose=0)
print(f'Hasil Prediksi: {CLASS_NAMES[np.argmax(classes)]}')